### End to end

Process


- Load neccessary data
- Repeat following 10 times
- Randomly select OAs and POIs to form an area of study
- Randomly select a weekday, weekend, bank holiday from period of study
- Cost trips on OTP (and OTP2 / r5?)
- Run matrix spasification techniques
    - knn
    - spatial clustering
    - flow clustering
    - gravity trip generator
- Reduce results as much as possible without loss of information

Module imports

todo: check to see if any unneccsary

In [19]:
import pandas as pd
import geopandas as gpd
from shapely import Point
from datetime import datetime, timedelta
import random
import numpy as np
from math import radians, cos, sin, asin, sqrt,exp,pow
import math
import statistics
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.cluster import HDBSCAN
import time
import matplotlib.pyplot as plt

def distance_decay(distance, decay_constant, exponent):
    return exp(-decay_constant * pow(distance, exponent))

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the haversine distance between two points given their latitude and longitude.

    Parameters:
    - lat1, lon1: Latitude and longitude of the first point (in degrees)
    - lat2, lon2: Latitude and longitude of the second point (in degrees)

    Returns:
    The haversine distance in kilometers.
    """
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Radius of the Earth in kilometers (mean value)
    earth_radius = 6371.0

    # Calculate the haversine distance
    distance = earth_radius * c

    return distance

def flowProx(flow_i,flow_j, alpha=1, beta=1, gamma=1):
    dist = (alpha*((flow_i[0] - flow_j[0]) ** 2) + beta*(((flow_i[2] - flow_j[2]) ** 2) + ((flow_i[3] - flow_j[3]) ** 2))) / ((haversine_distance(flow_i[0],flow_i[1],flow_i[2],flow_i[3]) * haversine_distance(flow_j[0],flow_j[1],flow_j[2],flow_j[3]))**gamma)
    return dist


def getminreach(xi,xj):
    flow_i=tuple([xi[0],xi[1],xi[2],xi[3]])
    flow_j=tuple([xj[0],xj[1],xj[2],xj[3]])
    CDi = xi[4]
    CDj = xj[4]
    prox_dist = flowProx(flow_i,flow_j, alpha=1, beta=1, gamma=1)
    mreachd = max(CDi,CDj,prox_dist)
    return mreachd

def get_flow_dist_mx(oaSample,POISample,minflows,flowProx):
    flows_index = []
    o_index = []
    d_index = []
    flows = []

    oind = 0
    for o in oaSample[['oa_lat','oa_lon','oa_id']].values:
        dind = 0
        for d in POISample[['poi_lon','poi_lat','poi_id']].values:
            flows.append(tuple([o[0], o[1], d[0], d[1]]))
            flows_index.append(tuple([o[2],int(d[2])]))
            o_index.append(o[2])
            d_index.append(int(d[2]))
            dind += 1
        oind += 1

    # Calculate CoreD
    # Create a NearestNeighbors object with a custom distance function
    neighbors_model = NearestNeighbors(n_neighbors=minflows, algorithm='ball_tree', metric=flowProx)
    X = np.array(flows)
    # Fit the model to your data
    neighbors_model.fit(X)

    core_distances = []
    count = 0
    for i in X:
        distances, indices = neighbors_model.kneighbors([i])
        core_distances.append(distances[:, -1][0])

    lst_reshaped = np.array(core_distances).reshape((len(core_distances), 1))
    return np.concatenate((X, lst_reshaped), axis=1), o_index, d_index, flows, flows_index

Set up code paramaters

In [3]:
# Get Time Stamps / Time Interval

stratumDict = {
    'amPeak':{
        'startHour' : 6,
        'startMinute' : 30,
        'endHour' : 8,
        'endMinute' : 30
        },
    'Saturday':{
        'startHour' : 10,
        'startMinute' : 00,
        'endHour' : 18,
        'endMinute' : 00
        },
    'bh':{
        'startHour' : 10,
        'startMinute' : 00,
        'endHour' : 18,
        'endMinute' : 00
        }
    }

#Params
decay_constant = 1
exponent = 0.2
#Integer for 1 trip per x minutes. E.g., 1 = a trip every minutes. 5 = 1 trip every 5 minutes
# For now go with every minutes but may pull this back
# To do - revisit later on
temp_resolution = 1

num_iterations = 10

num_oas = 200
num_pois = 50

# Vectrise distance decay function
vfunc = np.vectorize(distance_decay)

Import data


In [4]:
# Get day index
# Specify the start and end dates
start_date = '2024-03-15'
end_date = '2024-04-15'

# Create a date-time index
date_index = pd.date_range(start=start_date, end=end_date, freq='D')

experiment_dates = pd.DataFrame(index = date_index)

experiment_dates['weekday'] = experiment_dates.index.weekday < 5
experiment_dates['saturday'] = experiment_dates.index.weekday == 5
bank_holidays = ['2024-03-29', '2024-04-01']
experiment_dates['bank_holiday'] = experiment_dates.index.isin(pd.to_datetime(bank_holidays))

# Get OAs
wm_oas = gpd.read_file('data/west_midlands_OAs/west_midlands_OAs.shp')
wm_oas = wm_oas[wm_oas['LAD11CD'] == 'E08000026']
oa_info = pd.read_csv('data/oa_info.csv')
oa_info = oa_info.merge(wm_oas[['OA11CD']], left_on = 'oa_id', right_on = 'OA11CD', how = 'inner')
oaLatLon = oa_info[['oa_id','oa_lon','oa_lat']]

# Get POIs
pois = pd.read_csv('data/POIs/pois_cov.csv', index_col=0)

# Generate POI attractiveness using gaussian randomness

mean = 50
std_dev = 20
attractivnessDict = {}

for pid in list(pois['poi_id']):
    random_value = random.normalvariate(mean, std_dev)
    attractivnessDict[pid] = max(min(random_value, 100), 0)

Iterate

In [5]:
# Create tracking matrices

performance = {}
processing_times = {}
for i in range(num_iterations):
    performance[i] = {}
    processing_times[i] = {}
    for k in list(stratumDict.keys()):
        performance[i][k] = {}
        processing_times[i][k] = {}

In [6]:
#To do - decide number of iterations. What is easily defensible? 10?

iteration = 1

For an iteration select random OAs and POI - get from old code don't need to re-invent this

In [7]:
#Sample 200 random zones
oaSample = oa_info.sample(num_oas)[['oa_id','oa_lat','oa_lon']]

#POIs
POISample = pois.sample(num_pois)
POISample['attractiveness'] = POISample['poi_id'].map(attractivnessDict)

Define time domain


to do:

- Update to dynamically select at correct temporal granularity

In [8]:
stratum = 'amPeak'

if stratum == 'amPeak':
    study_date = experiment_dates[(experiment_dates['weekday']) & (experiment_dates['bank_holiday'] == False)].sample(1).index

elif stratum == 'Saturday':
    study_date = experiment_dates[(experiment_dates['saturday']) & (experiment_dates['bank_holiday'] == False)].sample(1).index

elif stratum == 'bh':
    study_date = experiment_dates[experiment_dates['bank_holiday']].sample(1).index

# Create Time Domain
startHour = stratumDict[stratum]['startHour']
startMinute = stratumDict[stratum]['startMinute']
endHour = stratumDict[stratum]['endHour']
endMinute = stratumDict[stratum]['endMinute']

start = datetime(year=2012, month=2, day=25, hour=startHour, minute = startMinute)
end = datetime(year=2012, month=2, day=25, hour=endHour, minute = endMinute)
diff = end - start
minutesInInterval = diff.total_seconds()/60
hoursInInterval = minutesInInterval/60

num_trips = int((60 / temp_resolution) * hoursInInterval)

timeDomain = []

for i in range(num_trips):
    randStartTime = start + timedelta(minutes=random.randint(1, int(minutesInInterval)))
    if randStartTime not in timeDomain:
        timeDomain.append(str(randStartTime.hour).zfill(2)+':'+str(randStartTime.minute).zfill(2))

In [ ]:
# Output trips to CSV

temp_trips_file = 'tempdata/trips_to_route.csv'
output_file = open(temp_trips_file, 'w')
writer = csv.writer(output_file)
writer.writerow(['oa_id','poi_id','trip_id','date','time','oa_lat','oa_lon','poi_lat','poi_lon'])

#Output trips dataset - output csv with following: 
trip_id = 0
#trip_date = study_date[0].strftime('%m/%d/%Y')
trip_date = study_date[0].strftime('%Y-%m-%d')

for oind, orow in oaSample.iterrows():
    for pind,prow in POISample.iterrows():
        for t in timeDomain:            
            row = [orow['oa_id'],prow['poi_id'],trip_id,trip_date,t,orow['oa_lat'], orow['oa_lon'],prow['poi_lat'], prow['poi_lon']]
            writer.writerow(row)
            trip_id += 1

In [ ]:
# Cost trips on OTP using parallelisation

num_trips = num_rows(temp_trips_file)
step_size = get_step_size(num_trips, processes)

args = []
for i in range(processes):
    host_url = f"http://{host}:{str(port + (i % otps))}"
    offset =i * step_size
    arg = (
        host_url, 
        offset, 
        min(offset+step_size, num_trips), 
        temp_trips_file, 
        'tempdata'
    )
    args.append(arg)

rows_complete = multiprocessing.Value('i', 0)

t0_routing_cost = time.time()
with multiprocessing.Pool(int(processes), initializer=init, initargs=(num_trips, rows_complete)) as pool:
    results = pool.starmap(compute_trips, args)
t1_routing_cost = time.time()
print('Time Taken : {}'.format(t1_routing_cost - t0_routing_cost))

files = []
bad_rows = 0
for f, rows in results:
    files.append(f)
    bad_rows += rows
if bad_rows > 0:
    print(f"{bad_rows} trips were lost during OTP processing ({(bad_rows/num_trips * 100):.2f}%).")

In [ ]:
# Output to CSV (needed)

output_file_name = os.path.join('tempdata', 'results_full.csv')
files_iterator = iter(files)
# Treat the first file differently, so we can extract headers
first_file = files[0]
headers = extract_headers(first_file)
with open(output_file_name, 'w') as output_file:
    output_csv = csv.writer(output_file)
    output_csv.writerow(headers)
    for csv_file in files:
        with open(csv_file, newline='') as f:
            reader = csv.DictReader(f)
            for line in reader:
                output_csv.writerow(line.values())

To do

- Develop costing script to either import here, or to be called with a parameter set
- Send trip to costing script and ensure returns correct data type
- Below probably to be removed

In [8]:
import csv
import os, urllib, json, csv, zipfile, math
import time

In [9]:
print('Num Rows : {}'.format(len(oaSample) * len(POISample) * len(timeDomain)))

otp_url = 'http://localhost:8080/otp/routers/default/plan?'
output_file = open('tempdata/tripscosts_otp.csv', 'w')
writer = csv.writer(output_file)
writer.writerow(['trip_id','date','time','oa_id','poi_id','itiniery_id','duration','walk_time','wait_time','transit_time','initial_wait_time','transfers','fare','routing_cost'])

trip_id = 0
itin_id = 0
trip_date = study_date[0].strftime('%m/%d/%Y')

failed_routes_list = []

for oind, orow in oaSample.iterrows():
    for pind,prow in POISample.iterrows():
        for t in timeDomain:
            if trip_id % 500 == 0:
                print(trip_id)
                
            params = {}
            params['date'] = trip_date
            params['time'] = t
            params['fromPlace'] = '%s,%s' % (orow['oa_lat'], orow['oa_lon'])
            params['toPlace'] = '%s,%s' % (prow['poi_lat'], prow['poi_lon'])
            params['mode'] = 'WALK,TRANSIT'
            params['arriveBy'] = 'false'
            params['numItineraries'] = '1'
            params['walkReluctance'] = '20'

            t0 = time.time()
            req = urllib.request.Request(otp_url + urllib.parse.urlencode(params))
            req.add_header('Accept', 'application/json')
            response = urllib.request.urlopen(req)
            content = response.read()
            objs = json.loads(content)
            t1 = time.time()

            if len(objs['plan']['itineraries']) == 0:
                fail_append = {}
                fail_append['oa'] = orow['oa_id']
                fail_append['poi'] = prow['poi_id']
                fail_append['time'] = t
                failed_routes_list.append(fail_append)
                pass
            else:
                i = objs['plan']['itineraries'][0]

            if i['transitTime'] == 0:
                fare = 0
            else:
                fare = (i['transfers'] + 1) * 2.4

            query_time = datetime.strptime(t, '%H:%M').time()
            departure_time = datetime.fromtimestamp(float(i['startTime']) / 1000).time()
            initial_wait_time = (datetime.combine(datetime.today(), departure_time) - datetime.combine(datetime.today(), query_time)).total_seconds()

            row = [trip_id,trip_date,t,orow['oa_id'],prow['poi_id'],itin_id,i['duration'],i['walkTime'],i['waitingTime'],i['transitTime'],initial_wait_time,i['transfers'],fare,(t1-t0)]
            writer.writerow(row)

            trip_id += 1

output_file.close()
failed_routes = pd.DataFrame(failed_routes_list)

Num Rows : 45000
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500


In [9]:
initial_trips = pd.read_csv('tempdata/trips_to_route.csv').set_index('trip_id')
trips = pd.read_csv('tempdata/results_full.csv').set_index('trip_id')
trips = trips.merge(initial_trips[['poi_id','oa_id','time']],right_index=True, left_index=True)

Once testing done drop below script

In [10]:
#Sample 200 random zones
oaSample = oa_info[oa_info['oa_id'].isin(list(set(trips['oa_id'])))][['oa_id','oa_lat','oa_lon']]

#POIs
POISample = pois[pois['poi_id'].isin(list(set(trips['poi_id'])))]
POISample['attractiveness'] = POISample['poi_id'].map(attractivnessDict)

/tmp/ipykernel_63663/1142294112.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['attractiveness'] = POISample['poi_id'].map(attractivnessDict)


In [11]:
# Create dict type:
    # key OAid - dict
        # key POIid - list
            # List of trip IDs

oa_poi_id_dict = {}

for oa_id in list(oaSample['oa_id']):
    oa_poi_id_dict[oa_id] = {}
    for poi_id in list(POISample['poi_id']):
        oa_poi_id_dict[oa_id][poi_id] = list(trips[(trips['oa_id'] == oa_id) & (trips['poi_id'] == poi_id)].index)

In [13]:
peformance_mx = pd.DataFrame(index = list(oaSample['oa_id']))
costing_mx = pd.DataFrame(index = list(oaSample['oa_id']))
computing_times = {}

In [14]:
#Compute generalised access cost
trips['gac'] = (( 1.5 * (trips['total_time'])) - (0.5 * trips['transit_time']) + ((trips['fare'] * 3600) / 6.7) + (10 * trips['num_transfers'])) / 60
trips['att'] = trips['poi_id'].map(attractivnessDict)

#Gravity Model Ground Truth
trips['dist decay'] = vfunc(np.array(trips['gac']),decay_constant, exponent)
trips['grav'] = trips['dist decay'] * trips['att']
gravity = trips.groupby('oa_id').sum()['grav']
print('Gravity model run')

peformance_mx['GM'] = gravity
costing_mx['GM'] = trips.groupby('oa_id').sum()['queryTime']

Gravity model run


Get Results KNN

To do

- Which values of k? Maybe just 5, 15, 25
- Collect metrics in tracking dicts
- Add time trap

In [14]:
for k in [5,15,25]:

    processing_time = 0

    t0 = time.time()

    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(POISample[['poi_lon','poi_lat']].values)
    kResList = []
    countOrigins = 0

    t1 = time.time()
    processing_time += (t1 - t0)

    for oind, orow in oaSample.iterrows():
        t0 = time.time()
        distances, indices = knn.kneighbors(orow[['oa_lon','oa_lat']].values.reshape(1, -1))
        t1 = time.time()
        processing_time += (t1 - t0)
        
        oCosts = []
        oTrips = 0
        oTimes = 0

        for i in indices[0]:
            pid = POISample.iloc[i]['poi_id']
            #trips_sample = trips[(trips['oa_id'] == orow['oa_id']) & (trips['poi_id'] == pid)]
            trips_sample = trips.loc[oa_poi_id_dict[orow['oa_id']][pid]]
            oCosts = oCosts + list(trips_sample['gac'])
            oTrips += len(trips_sample)
            oTimes += trips_sample['queryTime'].sum()

        kResAppend = {}
        kResAppend['oa_id'] = orow['oa_id']
        kResAppend['score'] = statistics.mean(oCosts)
        kResAppend['times'] = oTimes
        kResList.append(kResAppend)

    knnres = pd.DataFrame(kResList).set_index('oa_id')

    peformance_mx['knn_{}'.format(k)] = knnres['score']
    costing_mx['KNN_{}'.format(k)] = knnres['times']
    computing_times['KNN_{}'.format(k)] = processing_time

kmeans

In [42]:
for num_clusters in [3,4,5,6,7,8,9,10]:

    print('Next Test : {}'.format(num_clusters))

    processing_time = 0

    t0 = time.time()
    kmeans = KMeans(n_clusters=num_clusters, random_state=0, n_init="auto").fit(POISample[['poi_lon','poi_lat']].values)
    # Get cluster labels and centroids
    cluster_labels = kmeans.labels_
    t1 = time.time()
    processing_time += (t1 - t0)

    POISample['cluster'] = cluster_labels

    flows_df_list = []

    for cluster in set(cluster_labels):
        t0 = time.time()
        points_in_cluster = np.where(cluster_labels == cluster)[0]
        centroid = [POISample[POISample['cluster'] == cluster]['poi_lon'].mean(),POISample[POISample['cluster'] == cluster]['poi_lat'].mean()]
        distances_to_centroid = np.linalg.norm(POISample[['poi_lon','poi_lat']].values[points_in_cluster] - POISample[['poi_lon','poi_lat']].values[points_in_cluster].mean(axis = 0), axis=1)
        # Find the index of the point closest to the centroid
        closest_point_index = points_in_cluster[np.argmin(distances_to_centroid)]
        flow_id = POISample.iloc[closest_point_index]['poi_id']
        t1 = time.time()
        processing_time += (t1 - t0)
        oa_count = 0
        for oa in list(oaSample['oa_id']):
            oa_count += 1
            flow_gac = trips.loc[oa_poi_id_dict[oa][flow_id]][['oa_id','poi_id','time','gac','queryTime']]
            query_times = list(flow_gac['queryTime'])

            for poi in points_in_cluster:
                flow_append = flow_gac.copy()
                if poi != closest_point_index:
                    flow_append['queryTime'] = 0
                else:
                    flow_append['queryTime'] = query_times
                flow_append['poi_id'] = POISample.iloc[poi]['poi_id']
                flows_df_list.append(flow_append)

    flows_df = pd.concat(flows_df_list, ignore_index=True)
    flows_df['att'] = flows_df['poi_id'].map(attractivnessDict)
    flows_df['dist decay'] = vfunc(np.array(flows_df['gac']),decay_constant, exponent)
    flows_df['grav'] = flows_df['dist decay'] * flows_df['att']
    peformance_mx['kmean_{}'.format(num_clusters)] = flows_df.groupby('oa_id').sum()['grav']
    costing_mx['kmean_{}'.format(num_clusters)] = flows_df.groupby('oa_id').sum()['queryTime']
    computing_times['kmean_{}'.format(num_clusters)] = processing_time

Next Test : 3


/tmp/ipykernel_960/3469982662.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


Next Test : 4


/tmp/ipykernel_960/3469982662.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


Next Test : 5


...

/tmp/ipykernel_960/3469982662.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


Next Test : 9


/tmp/ipykernel_960/3469982662.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


Next Test : 10


/tmp/ipykernel_960/3469982662.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


DBSCAN

In [93]:
eps_test = [0.01,0.02,0.03]
min_samples = [1,3,5]

for ep in eps_test:
    for ms in min_samples:
        processing_time = 0

        t0 = time.time()
        dbscan = DBSCAN(eps=ep, min_samples=ms).fit(POISample[['poi_lon','poi_lat']].values)

        # Get cluster labels and centroids
        cluster_labels = dbscan.labels_
        POISample['cluster'] = cluster_labels
        t1 = time.time()
        processing_time += (t1 - t0)
        flows_df_list = []

        for cluster in set(cluster_labels):
            if cluster == -1:
                for c_ind in np.where(cluster_labels == cluster)[0]:
                    flow_id = POISample.iloc[c_ind]['poi_id']

                    for oa in list(oaSample['oa_id']):
                        flow_gac = trips.loc[oa_poi_id_dict[oa][flow_id]][['oa_id','poi_id','time','gac','queryTime']]
                        flows_df_list.append(flow_gac)
            else:
                t0 = time.time()
                points_in_cluster = np.where(cluster_labels == cluster)[0]
                centroid = [POISample[POISample['cluster'] == cluster]['poi_lon'].mean(),POISample[POISample['cluster'] == cluster]['poi_lat'].mean()]
                distances_to_centroid = np.linalg.norm(POISample[['poi_lon','poi_lat']].values[points_in_cluster] - POISample[['poi_lon','poi_lat']].values[points_in_cluster].mean(axis = 0), axis=1)
                # Find the index of the point closest to the centroid
                closest_point_index = points_in_cluster[np.argmin(distances_to_centroid)]
                flow_id = POISample.iloc[closest_point_index]['poi_id']
                t1 = time.time()
                processing_time += (t1 - t0)
                for oa in list(oaSample['oa_id']):
                    flow_gac = trips.loc[oa_poi_id_dict[oa][flow_id]][['oa_id','poi_id','time','gac','queryTime']]
                    query_times = list(flow_gac['queryTime'])
                    for poi in points_in_cluster:
                        flow_append = flow_gac.copy()
                        if poi != closest_point_index:
                            flow_append['queryTime'] = 0
                        else:
                            flow_append['queryTime'] = query_times
                        flow_append['poi_id'] = POISample.iloc[poi]['poi_id']
                        flows_df_list.append(flow_append)

        flows_df = pd.concat(flows_df_list, ignore_index=True)
        flows_df['att'] = flows_df['poi_id'].map(attractivnessDict)
        flows_df['dist decay'] = vfunc(np.array(flows_df['gac']),decay_constant, exponent)
        flows_df['grav'] = flows_df['dist decay'] * flows_df['att']
        peformance_mx['dbscan_{}_{}'.format(ep,ms)] = flows_df.groupby('oa_id').sum()['grav']
        costing_mx['dbscan_{}_{}'.format(ep,ms)] = flows_df.groupby('oa_id').sum()['queryTime']
        computing_times['dbscan_{}_{}'.format(ep,ms)] = processing_time

0.01
1


/tmp/ipykernel_960/336290671.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


0.01
3


/tmp/ipykernel_960/336290671.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


0.01
5


...

/tmp/ipykernel_960/336290671.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


0.03
3


/tmp/ipykernel_960/336290671.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


0.03
5


/tmp/ipykernel_960/336290671.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


HDBSCAN

In [113]:
min_clusters = [3,5,7]
min_sample_tests = [1,3,5,7]

for mc in min_clusters:
    for ms in min_sample_tests:

        processing_time = 0

        t0 = time.time()
        hdbscan = HDBSCAN(min_cluster_size = mc, min_samples=ms).fit(POISample[['poi_lon','poi_lat']].values)

        # Get cluster labels and centroids
        cluster_labels = hdbscan.labels_
        POISample['cluster'] = cluster_labels
        t1 = time.time()
        processing_time += (t1 - t0)
        print('Cluster Labels : {}'.format(cluster_labels))

        flows_df_list = []

        for cluster in set(cluster_labels):
            if cluster == -1:
                for c_ind in np.where(cluster_labels == cluster)[0]:
                    flow_id = POISample.iloc[c_ind]['poi_id']

                    for oa in list(oaSample['oa_id']):
                        flow_gac = trips.loc[oa_poi_id_dict[oa][flow_id]][['oa_id','poi_id','time','gac','queryTime']]
                        flows_df_list.append(flow_gac)
            else:
                t0 = time.time()
                points_in_cluster = np.where(cluster_labels == cluster)[0]
                centroid = [POISample[POISample['cluster'] == cluster]['poi_lon'].mean(),POISample[POISample['cluster'] == cluster]['poi_lat'].mean()]
                distances_to_centroid = np.linalg.norm(POISample[['poi_lon','poi_lat']].values[points_in_cluster] - POISample[['poi_lon','poi_lat']].values[points_in_cluster].mean(axis = 0), axis=1)
                # Find the index of the point closest to the centroid
                closest_point_index = points_in_cluster[np.argmin(distances_to_centroid)]
                flow_id = POISample.iloc[closest_point_index]['poi_id']
                t1 = time.time()
                processing_time += (t1 - t0)
                for oa in list(oaSample['oa_id']):
                    flow_gac = trips.loc[oa_poi_id_dict[oa][flow_id]][['oa_id','poi_id','time','gac','queryTime']]
                    query_times = list(flow_gac['queryTime'])
                    for poi in points_in_cluster:
                        flow_append = flow_gac.copy()
                        if poi != closest_point_index:
                            flow_append['queryTime'] = 0
                        else:
                            flow_append['queryTime'] = query_times
                        flow_append['poi_id'] = POISample.iloc[poi]['poi_id']
                        flows_df_list.append(flow_append)

        flows_df = pd.concat(flows_df_list, ignore_index=True)
        flows_df['att'] = flows_df['poi_id'].map(attractivnessDict)
        flows_df['dist decay'] = vfunc(np.array(flows_df['gac']),decay_constant, exponent)
        flows_df['grav'] = flows_df['dist decay'] * flows_df['att']
        peformance_mx['hdbscan_{}_{}'.format(mc,ms)] = flows_df.groupby('oa_id').sum()['grav']
        costing_mx['hdbscan_{}_{}'.format(mc,ms)] = flows_df.groupby('oa_id').sum()['queryTime']
        computing_times['hdbscan_{}_{}'.format(mc,ms)] = processing_time

/tmp/ipykernel_960/3774843614.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


Cluster Labels : [ 3  3  7  1 -1  3  5  5  7  4  2  6  6  2  7  4 -1  0  1  6  3  6  2  2
  2  2  1  2 -1 -1  5  5  5  7  5  7  7  7  5  6  6  6  6 -1  4  0  7  6
  0  7]


/tmp/ipykernel_960/3774843614.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


Cluster Labels : [ 1  1  6  2 -1  1  4  4  5  1 -1  5 -1  3  6  1 -1  0  2  5  1  5  3  3
  3  3  2  3 -1 -1  4  4  4 -1  4  6 -1  6 -1  5  5  5  5 -1  1  0  6  5
  0  6]


/tmp/ipykernel_960/3774843614.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


...

Cluster Labels : [ 0  0  0  1  1  0  0  0  0  0  1  0  0  1  0  0 -1  0  1  0  0  0  1  1
  1  1  1  1  1  0  0  0  0  0  0  0 -1  0  0  0  0  0  0 -1  0  0  0  0
  0  0]


/tmp/ipykernel_960/3774843614.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


Cluster Labels : [ 0  0  0  1  1  0  0  0  0  0  1  0  0  1  0  0 -1  0  1  0  0  0  1  1
  1  1  1  1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0 -1  0 -1  0  0
  0  0]


/tmp/ipykernel_960/3774843614.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POISample['cluster'] = cluster_labels


Cluster Labels : [ 0  0  0  1  1  0  0  0  0  0  1  0  0  1  0  0 -1 -1  1  0  0  0  1  1
  1  1  1  1  0 -1 -1  0  0  0  0  0 -1 -1  0  0  0  0  0 -1  0 -1  0  0
 -1  0]


Get results flow clusering

To do

- Function to get distace matrix
- Test correlation over a set of parameters
- Collect metrics
- Time traps

HDBSCAN with flow

In [74]:
# Get flow
# Cluster flows

minflows = [3,5,7]
minclusters = [5,10,15]
minsamples = [5,10,15]
        
for mf in minflows:
    X, o_index, d_index, flows, flows_index = get_flow_dist_mx(oaSample,POISample,mf,flowProx)
    for mc in minclusters:
        for ms in minsamples:
            
            processing_time = 0

            t0 = time.time()
            hdb = HDBSCAN(min_cluster_size=mc, min_samples=ms, metric=getminreach).fit(X)
            cluster_labels = hdb.labels_
            t1 = time.time()
            processing_time += (t1 - t0)
            print(cluster_labels)

            flows_df_list = []
            for cluster in set(cluster_labels):
                if cluster == -1:
                    cluster_indeces = np.where(cluster_labels==cluster)[0]
                    flow_inds = list(set([flows_index[i] for i in cluster_indeces]))
                    for f in flow_inds:
                        flow_gac = trips.loc[oa_poi_id_dict[f[0]][f[1]]][['oa_id','poi_id','time','gac','queryTime']]
                        flows_df_list.append(flow_gac)
                else:
                    t0 = time.time()
                    cluster_indeces = np.where(cluster_labels==cluster)[0]
                    #Select origin of best flow
                    oas_in_flow = list(set([o_index[i] for i in cluster_indeces]))
                    distances_to_centroid = np.linalg.norm(oaSample.set_index('oa_id').loc[oas_in_flow].values - oaSample.set_index('oa_id').loc[oas_in_flow].values.mean(axis = 0),axis=1)
                    flow_oa = oas_in_flow[np.argmin(distances_to_centroid)]
                    #Select destination of best flow
                    poi_ids = list(set([d_index[i] for i in cluster_indeces]))
                    distances_to_centroid = np.linalg.norm(POISample.set_index('poi_id').loc[poi_ids][['poi_lon','poi_lat']].values - POISample.set_index('poi_id').loc[poi_ids][['poi_lon','poi_lat']].values.mean(axis = 0),axis = 1)
                    flow_poi = poi_ids[np.argmin(distances_to_centroid)]
                    t1 = time.time()
                    processing_time += (t1 - t0)
                    #measure GAC for all time steps
                    flow_gac = trips.loc[oa_poi_id_dict[flow_oa][flow_poi]][['oa_id','poi_id','time','gac','queryTime']]
                    query_times = list(flow_gac['queryTime'])
                    flow_inds = list(set([flows_index[i] for i in cluster_indeces]))
                    for f in flow_inds:
                        flow_append = flow_gac.copy()
                        if f[1] != flow_poi:
                            flow_append['queryTime'] = 0
                        else:
                            flow_append['queryTime'] = query_times
                        flow_append['oa_id'] = f[0]
                        flow_append['poi_id'] = f[1]
                        flows_df_list.append(flow_append)
            flows_df = pd.concat(flows_df_list, ignore_index=True)
            flows_df['att'] = flows_df['poi_id'].map(attractivnessDict)
            flows_df['dist decay'] = vfunc(np.array(flows_df['gac']),decay_constant, exponent)
            flows_df['grav'] = flows_df['dist decay'] * flows_df['att']
            peformance_mx['flowhdbscan_{}_{}_{}'.format(mf,mc,ms)] = flows_df.groupby('oa_id').sum()['grav']
            costing_mx['flowhdbscan_{}_{}_{}'.format(mf,mc,ms)] = flows_df.groupby('oa_id').sum()['queryTime']
            computing_times['flowhdbscan_{}_{}_{}'.format(mf,mc,ms)] = processing_time

[161  88 274 ... 762 639 756]
[ 56  32  59 ... 430 362 440]
[ -1  77  66 ... 107  27  88]
[ 38  20  80 ... 330 198 337]
[ 65  34  68 ... 148  57 141]
[-1 76 68 ... 87 14 69]
[ 38  19  76 ... 322 190 329]
[85 57 92 ... 93  7 89]
[94 76 68 ... 87 14 69]
[162  73 210 ... 692 409 691]
[ 47  31  58 ... 411 339 429]
[-1 71 63 ... 98 13 87]
[ 40  20  85 ... 389 180 383]
[ 55  33  66 ... 137  44 145]
[-1 74 70 ... 87 18 67]
[ 34  18  76 ... 333 146 327]
[70 55 93 ... 86  7 90]
[94 74 70 ... 87 18 67]
[165 222 276 ... 705 624 673]
[ 79  38  62 ... 341 292 369]
[-1 81 51 ... 98 35 79]
[105 113 158 ... 352 196 333]
[109  48  78 ...  94  59 134]
[-1 76 56 ... 87 12 53]
[31 28 38 ... 42  2 37]
[93 61 89 ... 72  3 74]
[94 76 56 ... 87 12 53]


Get Results trip generator

In [ ]:
#METHOD 1 - Trip Generator (Gravity)

distMxList = []

for i,r in oas.iterrows():
    for i_, r_ in pois.iterrows():
        rowAppend = {}
        rowAppend['oa'] = r['oa_id']
        rowAppend['poi'] = r_['poi_id']
        rowAppend['dist'] = haversine(r['oa_lon'], r['oa_lat'], r_['poi_lon'], r_['poi_lat'])
        distMxList.append(rowAppend)

distMx = pd.DataFrame(distMxList)

distMx['att'] = distMx['poi'].map(attractivnessDict)

distsDecay = []
for i in np.array(distMx['dist']):
    distsDecay.append(distance_decay(i, decay_constant, exponent))

distMx['decay'] = distsDecay
distMx['grav'] = distMx['decay'] * distMx['att']
distMx['gravN'] = (distMx['grav'] - distMx['grav'].min()) / (distMx['grav'].max() - distMx['grav'].min())
distMx = distMx.merge(tripsL.groupby(['oa_id','poi_id']).count()['departure_time'].rename('tripCount'), left_on = ['oa','poi'],right_index = True)
distMx['tripsSample'] = distMx['tripCount'] * distMx['gravN']

resultsList = []

countOrigins = 0

print('Running Trip Gen Gravity')

for o in oaIndex:
    countOrigins += 1
    if countOrigins % 20 == 0:
        print(countOrigins)
    oCosts = []
    oTrips = 0
    oTimes = 0
    for p in poiIndex:
    
        numSample = int(distMx[(distMx['oa'] == o) & (distMx['poi'] == p)]['tripsSample'].values[0])
        tripSample = tripsL[(tripsL['oa_id'] == o) & (tripsL['poi_id'] == p)].sample(numSample)
        oCosts = oCosts + list(tripSample[cost])
        oTrips += numSample
        oTimes += tripSample['queryTime'].sum()

    
    rowAppend = {}
    rowAppend['OA'] = o
    rowAppend['GTG'] = statistics.mean(oCosts)
    rowAppend['GTG_Trips'] = oTrips
    rowAppend['GTG_Times'] = oTimes
    resultsList.append(rowAppend)

tripGenGrav = pd.DataFrame(resultsList)
zoneLevelResults = tripGenGrav.merge(gravity, left_on = 'OA', right_index = True)

print('Fnished trip gen grav')

To do:

- Any optimisations needed?
- Add in code from previous version
- Add to metrics
- Add time trap

General to dos after script finished

- Put into single python script and test locally on small number of OAs and POIs
- Script parameters
    - Number of OAs
    - Number of POIs
    - Temporal granularity
    - Parallelisation metrics
- Carefully check all outputs as expected and initial analysis of results
- Set up server on digital ocean which can parallise over 30+ instaces
- Upload all data etc
- Run test script to check
    - Trip costing working e.g., GAC generating as expected (e.g., add trips output to observe)
    - Check parallisation works and does not break server
    - Check results are as expected
- Extend to run on full set of experiments
- Run observe for a while, predict how long it'll take and make sure not too costly